In [1]:
from util import *
import pyspark.sql.functions as f
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

initial_data_file = "C:/Users/parth/Desktop/workspace/data_pipeline/resources/sample_data/customers.csv"
updated_data_file = "C:/Users/parth/Desktop/workspace/data_pipeline/resources/sample_data/customers_full.csv"

output_path = "C:/Users/parth/Desktop/workspace/data_pipeline/output/customers_full_overwrite_history"
table_name = "customers_full_overwrite_history"
primary_key_field = "id"
parition_field = "date"
precombine_field = "updated_date"
spark_write_mode = "append"
parition_from = "updated_date"
hudi_options = get_incremental_options(table_name, primary_key_field, parition_field, precombine_field)
source_columns = ["id", "name", "email", "created_date", "updated_date"]
timestamp_columns = ["created_date", "updated_date"]
hudi_options

{'hoodie.table.name': 'customers_full_overwrite_history',
 'hoodie.datasource.write.recordkey.field': 'id',
 'hoodie.datasource.write.partitionpath.field': 'date',
 'hoodie.datasource.write.table.name': 'customers_full_overwrite_history',
 'hoodie.datasource.write.operation': 'upsert',
 'hoodie.datasource.write.precombine.field': 'updated_date'}

In [2]:
import os
import shutil

if os.path.exists(output_path):
    shutil.rmtree(output_path)

In [3]:
spark = get_spark_with_hudi()

In [4]:
source_df = spark.read.option("header", "true").option("inferSchema", "true").format("csv").load(initial_data_file)

for column in timestamp_columns:
    source_df = source_df.withColumn(column, f.col(column).cast("timestamp"))

In [5]:
w_create_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
processed_df = (source_df
    .withColumn("is_active", f.lit(True))
    .withColumn("effective_start_date", f.to_timestamp(f.lit(w_create_ts)))
    .withColumn("effective_end_date", f.to_timestamp(f.lit("2999-12-31 23:59:59"))))
processed_df.show()

+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+
| id|           name|               email|       created_date|       updated_date|is_active|effective_start_date| effective_end_date|
+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+
|  1|Jammie McCamish|jmccamish0@devhub...|2023-01-01 03:17:00|2023-01-01 03:17:00|     true| 2023-01-14 20:12:55|2999-12-31 23:59:59|
|  2| Brocky Spurret|   bspurret1@npr.org|2023-01-01 05:35:00|2023-01-01 05:35:00|     true| 2023-01-14 20:12:55|2999-12-31 23:59:59|
+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+



In [6]:
processed_df.withColumn("date", f.date_format(f.col(parition_from), "yyyyMMdd")).withColumn("w_create_ts", f.lit(w_create_ts).cast("timestamp")).write.format("hudi").options(**hudi_options).mode(spark_write_mode).save(output_path)

In [7]:
source_df = spark.read.option("header", "true").option("inferSchema", "true").format("csv").load(updated_data_file)

for column in timestamp_columns:
    source_df = source_df.withColumn(column, f.col(column).cast("timestamp"))

w_create_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

source_df = (source_df
    .withColumn("is_active", f.lit(True))
    .withColumn("effective_start_date", f.to_timestamp(f.lit(w_create_ts)))
    .withColumn("effective_end_date", f.to_timestamp(f.lit("2999-12-31 23:59:59"))))

In [8]:
prev_df = spark.read.format("hudi").load(output_path)
prev_df_intermediate = prev_df.select(*source_columns).exceptAll(source_df.select(*source_columns))
prev_df_incremental =(prev_df_intermediate
    .join(prev_df.select(primary_key_field, "effective_start_date", "effective_end_date", "is_active"), on=primary_key_field)
)
prev_df_incremental.show()

+---+---------------+--------------------+-------------------+-------------------+--------------------+-------------------+---------+
| id|           name|               email|       created_date|       updated_date|effective_start_date| effective_end_date|is_active|
+---+---------------+--------------------+-------------------+-------------------+--------------------+-------------------+---------+
|  1|Jammie McCamish|jmccamish0@devhub...|2023-01-01 03:17:00|2023-01-01 03:17:00| 2023-01-14 20:12:55|2999-12-31 23:59:59|     true|
+---+---------------+--------------------+-------------------+-------------------+--------------------+-------------------+---------+



In [9]:
join_condition = [prev_df_incremental[primary_key_field] == source_df[primary_key_field], prev_df_incremental["is_active"]]
to_update_df = (prev_df_incremental
    .join(source_df, on=join_condition)
    .select(prev_df_incremental["*"], source_df["effective_start_date"].alias("prev_effective_end_date"))
    .withColumn("effective_end_date", f.col("prev_effective_end_date"))
    .withColumn("is_active", f.lit(False))
    .drop("prev_effective_end_date"))
final_df = source_df.unionByName(to_update_df)
final_df.show()

+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+
| id|           name|               email|       created_date|       updated_date|is_active|effective_start_date| effective_end_date|
+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+
|  2| Brocky Spurret|   bspurret1@npr.org|2023-01-01 05:35:00|2023-01-01 05:35:00|     true| 2023-01-14 20:13:06|2999-12-31 23:59:59|
|  1|Jammie McCamish|jmccamish0@develo...|2023-01-01 03:17:00|2023-01-02 03:17:00|     true| 2023-01-14 20:13:06|2999-12-31 23:59:59|
|  1|Jammie McCamish|jmccamish0@devhub...|2023-01-01 03:17:00|2023-01-01 03:17:00|    false| 2023-01-14 20:12:55|2023-01-14 20:13:06|
+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+



In [10]:
final_df.withColumn("date", f.date_format(f.col(parition_from), "yyyyMMdd")).withColumn("w_create_ts", f.lit(w_create_ts).cast("timestamp")).write.format("hudi").options(**hudi_options).mode(spark_write_mode).save(output_path)

In [16]:
spark.read.format("hudi").load(output_path).select(*source_columns, "is_active", "effective_start_date", "effective_end_date").show()

+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+
| id|           name|               email|       created_date|       updated_date|is_active|effective_start_date| effective_end_date|
+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+
|  2| Brocky Spurret|   bspurret1@npr.org|2023-01-01 05:35:00|2023-01-01 05:35:00|     true| 2023-01-14 20:13:06|2999-12-31 23:59:59|
|  1|Jammie McCamish|jmccamish0@devhub...|2023-01-01 03:17:00|2023-01-01 03:17:00|    false| 2023-01-14 20:12:55|2023-01-14 20:13:06|
|  1|Jammie McCamish|jmccamish0@develo...|2023-01-01 03:17:00|2023-01-02 03:17:00|     true| 2023-01-14 20:13:06|2999-12-31 23:59:59|
+---+---------------+--------------------+-------------------+-------------------+---------+--------------------+-------------------+

